LOADING REQUIRED PACKAGES

In [ ]:
import sys
import tensorflow as tf

import os
import glob
import cv2
import random
import numpy as np

from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
print('packages installed')

LOAD A PRETRAINED MODEL

In [ ]:
# Load pre-trained U-Net model
pretrained_model = tf.keras.models.load_model('/Users/tebogoletshwiti/Desktop/unet_model_epoch_5000.h5')

# Display the summary of the pre-trained model
pretrained_model.summary()

SELECT THE FINAL OUTPUT OF THE ENCODER TO INPUT INTO THE DECODER

In [ ]:
# Get the output of the last layer in the encoder (decoder input)
decoder_input = pretrained_model.get_layer('conv2d_9').output

CREATE A NEW DECODER/EXPANSIVE PATH FOR THE MODEL

In [ ]:
#Expansive path
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='ex_layer_11')(decoder_input)
u6 = tf.keras.layers.concatenate([u6, pretrained_model.get_layer('conv2d_7').output])
c6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_12')(u6)
c6 = tf.keras.layers.Dropout(0.2, name='do_layer_1')(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_13')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2, 2), padding='same', name='ex_layer_14')(c6)
u7 = tf.keras.layers.concatenate([u7, pretrained_model.get_layer('conv2d_5').output])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_15')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_16')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, pretrained_model.get_layer('conv2d_3').output])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_17')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_18')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, pretrained_model.get_layer('conv2d_1').output], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_19')(u9)
c9 = tf.keras.layers.Dropout(0.1, name='do_layer_2')(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', name='ex_layer_20')(c9)

outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', name='ex_layer_21')(c9)
new_model = tf.keras.Model(inputs=pretrained_model.input, outputs=[outputs])
new_model.summary()

IDENTIFY THE INDEX OF THE LAYER WHERE YOU WANT TO SPLIT THE PRETRAINED MODEL

In [ ]:
# Identify the index of the layer where you want to split the pretrained model
split_layer_index = new_model.layers.index(pretrained_model.get_layer('conv2d_9'))

# Freeze the layers up to the split layer
for layer in new_model.layers[:split_layer_index]:
    layer.trainable = False

In [ ]:
# Compile the model (adjust based on your task)
new_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
new_model.summary()

LOAD IMAGES AND CREATE THE TRAINING AND TESTING DATASET

In [ ]:
# Folder containing all the images and masks
data_folder = '/Users/tebogoletshwiti/Desktop/Image_Segmentation'

# List of images and masks
# image_ids = next(os.walk(data_folder + '/images'))[2]
masks_ids = next(os.walk(data_folder + '/masks'))[2]

# divide images 80% to training and 20% to testing
training_data = []
testing_data = image_ids.copy()

file1 = open("/content/gdrive/MyDrive/Colab Notebooks/training_images.txt","a")
file2 = open("/content/gdrive/MyDrive/Colab Notebooks/testing_images.txt","a")

total = 0
while total <= (round(len(image_ids) * 0.8)):

  index = random.randint(0, len(testing_data) - 1)
  training_data.append(image_ids[index])
  testing_data.pop(index)

  total += 1

for image in training_data:
  file1.writelines(image + '\n')

for image in testing_data:
  file2.writelines(image + '\n')

file1.close()
file2.close()

In [ ]:
training_imgs = []
testing_imgs = []

data_folder = '/Users/tebogoletshwiti/Desktop/Image_Segmentation/'

file1 = open(data_folder + "training_images.txt","r+")
file2 = open(data_folder + "testing_images.txt","r+")

for item in file1.readlines():
  training_imgs.append(item.strip())

for item in file2.readlines():
  testing_imgs.append(item.strip())

mask_ids = training_imgs.copy()

In [ ]:
seed = 44
np.random.seed = seed

IMG_WIDTH = 640
IMG_HEIGHT = 640
IMG_CHANNELS = 3

X_train = np.zeros((len(training_imgs), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(training_imgs), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print('Resizing training images and masks')
for n, id_ in tqdm(enumerate(training_imgs), total=len(training_imgs)):

    img = cv2.imread(data_folder + 'images/' + id_)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_NEAREST)[:,:,:IMG_CHANNELS]
    X_train[n] = img  #Fill empty X_train with values from img

    mask_ = cv2.imread(data_folder + 'masks/' + id_[0:-4] + '.png')
    mask_ = cv2.resize(mask_, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_NEAREST)[:,:,:1]
    Y_train[n] = mask_

# test images
X_test = np.zeros((len(testing_imgs), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
print('Resizing test images')
for n, id_ in tqdm(enumerate(testing_imgs), total=len(testing_imgs)):
  img = cv2.imread(data_folder + 'images/' + id_)
  img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_NEAREST)[:,:,:IMG_CHANNELS]
  X_test[n] = img

RUN MODEL FIT TO TRAIN THE MODEL

In [ ]:
from keras.api._v2.keras import callbacks
#model Checkpoint

checkpointer = tf.keras.callbacks.ModelCheckpoint('Model_for_cracks.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_loss'),
#     tf.keras.callbacks.TensorBoard(log_dir='logs')
# ]

results = new_model.fit(X_train, Y_train, validation_split=0.2, batch_size=10, epochs=15)

RUN NODEL PREDICTION ON TESTING DATA

In [ ]:
preds_train = new_model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = new_model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = new_model.predict(X_test, verbose=1)

preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

RUN MODEL EVALUATION ON TEST DATA

In [ ]:
testing = np.zeros((len(testing_imgs), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)

test_results = new_model.evaluate(X_test, testing, callbacks=[tf.keras.callbacks.TensorBoard(log_dir="logs/evaluate/")])

print("test loss, test acc: ", test_results)


In [ ]:
Mean_IoU = []
con_matrix = []

for n, id_ in tqdm(enumerate(testing_imgs), total=len(testing_imgs)):

    img_mask = cv2.imread(data_folder + '/masks/' + testing_imgs[n][0:-4] + '.png')
    true_mask = cv2.resize(img_mask, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_NEAREST)[:, :, :1]/255
    true_mask_t = (true_mask > 0.4).astype(np.uint8)
    testing[n] = true_mask_t

    num_classes = 2
    IOU_keras = tf.keras.metrics.MeanIoU(num_classes=num_classes)
    IOU_keras.update_state(true_mask_t, preds_test_t[n])
    Mean_IoU.append(IOU_keras.result().numpy())


    values = np.array(IOU_keras.get_weights()).reshape(num_classes, num_classes)
    con_matrix.append(values)

In [ ]:
#Model metrics

Pr = []
Re = []
F1_score = []

for matrix in con_matrix:

    if (matrix[1, 1] + matrix[0, 1]) != 0:
        precision = matrix[1, 1] / (matrix[1, 1] + matrix[0, 1])

    else:
        precision = 0

    if (matrix[1, 1] + matrix[1, 0]) != 0:
        recall = matrix[1, 1] / (matrix[1, 1] + matrix[1, 0])

    else:
        recall = 0

    if (precision + recall) != 0:
        F1 = (2 * precision * recall) / (precision + recall)

    else:
        F1 = 0

    Pr.append(precision)
    Re.append(recall)
    F1_score.append(F1)

Ave_Pr = sum(Pr) / len(Pr)
Ave_Re = sum(Re) / len(Re)
Ave_F1 = sum(F1_score) / len(F1_score)

print('Precision is: ', Ave_Pr)
print('Recall is: ', Ave_Re)
print('F1 Score is: ', Ave_F1)
print('IoU is: ', sum(Mean_IoU) / len(Mean_IoU))